In [126]:
import time
import os
import urllib.parse

import numpy as np
import tensorflow as tf
from tensorflow.core.example.example_pb2 import Example
from tensorflow.core.example.feature_pb2 import Feature
from sklearn.model_selection import train_test_split
import pandas as pd

tf.logging.set_verbosity(tf.logging.INFO) 

# Get data
We are using the Census data. We are proposing a process to assess algorithmic bias in models, so instead of using the classification for whether people have >50k income, it will be used to classify if people should get a loan.

In [127]:
DOWNLOAD = False

csv_columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
                   "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
                   "hours-per-week", "native-country", "loan"]

label_column = 'loan'

csv_path = "data/adult.csv"

if DOWNLOAD:
    csv_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

# Read the dataset from the provided CSV and print out information about it.
df = pd.read_csv(csv_path, names=csv_columns, skiprows=[0], skipinitialspace=True)

# Converting Loan column to 1 if >50K
df[label_column] = df[label_column].apply(lambda x: ">50K" in x).astype(int)

# Delete unused column
del df["fnlwgt"] 

df.head(5)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,loan
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [128]:
model_folder = 'trained_model'

examples_file = 'data.tfrecord'

model_path = os.path.join(os.getcwd(), model_folder)
examples_path = os.path.join(os.getcwd(), examples_file)

In [129]:
X = df.drop(label_column, axis=1)
y = df[label_column]

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [131]:
BATCH_SIZE = 40
num_epochs = 1
shuffle = True

In [132]:
# Categorical base columns.
sex = tf.feature_column.categorical_column_with_vocabulary_list(
    key="sex",                                                           
    vocabulary_list=["female", "male"])
race = tf.feature_column.categorical_column_with_vocabulary_list(
    key="race",                                                             
    vocabulary_list=["Amer-Indian-Eskimo",
                     "Asian-Pac-Islander",
                     "Black", "Other", "White"])

education = tf.feature_column.categorical_column_with_hash_bucket(
  "education", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket(
  "marital-status", hash_bucket_size=100)
relationship = tf.feature_column.categorical_column_with_hash_bucket(
  "relationship", hash_bucket_size=100)
workclass = tf.feature_column.categorical_column_with_hash_bucket(
  "workclass", hash_bucket_size=100)
occupation = tf.feature_column.categorical_column_with_hash_bucket(
  "occupation", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket(
  "native-country", hash_bucket_size=1000)

In [133]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education-num")
capital_gain = tf.feature_column.numeric_column("capital-gain")
capital_loss  = tf.feature_column.numeric_column("capital-loss")
hours_per_week = tf.feature_column.numeric_column("hours-per-week")

In [138]:
all_columns = [
    tf.feature_column.indicator_column(workclass),
    tf.feature_column.indicator_column(marital_status),
    tf.feature_column.indicator_column(sex),
    tf.feature_column.indicator_column(relationship),
    tf.feature_column.indicator_column(race),
    tf.feature_column.embedding_column(education, dimension=8),
    tf.feature_column.embedding_column(native_country, dimension=8),
    tf.feature_column.embedding_column(occupation, dimension=8),
    age,
    education_num,
    capital_gain,
    capital_loss,
    hours_per_week,
]

In [139]:

model_dir = "models/model_DEEP_" + str(int(time.time()))
m = tf.estimator.DNNClassifier(
            model_dir=model_dir,
            feature_columns=all_columns,
            hidden_units=[100,100,100])



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/model_DEEP_1543760609', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffa31895e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [140]:
%%time 

train_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=X_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        num_epochs=num_epochs,
        shuffle=shuffle)

m.train(input_fn=train_input_fn)

print('training done')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into models/model_DEEP_1543760609/model.ckpt.
INFO:tensorflow:loss = 106.8656, step = 1
INFO:tensorflow:global_step/sec: 77.2932
INFO:tensorflow:loss = 12.86287, step = 101 (1.295 sec)
INFO:tensorflow:global_step/sec: 94.4765
INFO:tensorflow:loss = 20.765036, step = 201 (1.058 sec)
INFO:tensorflow:global_step/sec: 98.3633
INFO:tensorflow:loss = 16.84785, step = 301 (1.016 sec)
INFO:tensorflow:global_step/sec: 100.453
INFO:tensorflow:loss = 14.2874, step = 401 (0.995 sec)
INFO:tensorflow:global_step/sec: 102.983
INFO:tensorflow:loss = 13.47528, step = 501 (0.971 sec)
INFO:tensorflow:global_step/sec: 105.905
INFO:tensorflow:loss = 18.753456, step = 601 (0.944 sec)
INFO:tensorflow:global_step/sec: 106.444
INFO:tenso

In [141]:
eval_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=X_test,
        y=y_test,
        batch_size=BATCH_SIZE,
        num_epochs=num_epochs,
        shuffle=shuffle)

results = m.evaluate(input_fn=eval_input_fn)

print('\nAccuracy: %s' % results['accuracy'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-02-14:23:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/model_DEEP_1543760609/model.ckpt-977
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-02-14:23:44
INFO:tensorflow:Saving dict for global step 977: accuracy = 0.83580714, accuracy_baseline = 0.76384485, auc = 0.88996124, auc_precision_recall = 0.72008383, average_loss = 0.34758908, global_step = 977, label/mean = 0.23615518, loss = 13.859582, precision = 0.7714286, prediction/mean = 0.20284212, recall = 0.43302992
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 977: models/model_DEEP_1543760609/model.ckpt-977

Accuracy: 0.83580714


# Explainability with LIME